# Code Generation and Evaluation

In [1]:
from human_eval.data import write_jsonl, read_problems
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import random, time, datetime
from mistralai import Mistral
import json

%load_ext autoreload
%autoreload 2

In [2]:
from prompts import ( complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,
                      complete_task_prompt_basic, complete_task_prompt, complete_task_prompt_full,
                      reflection_prompt_basic, reflection_prompt, reflection_prompt_full, )
from helpers import clean_code_light, is_function            # write_jsonl

In [3]:
# select model, propmt, temperature, top_p
models = [
    ('ministral-3b-latest', 'mistral_3b'),
    ('ministral-8b-latest', 'mistral_8B'),
    ('open-mistral-nemo', 'nemo'),
    #('open-codestral-mamba', 'codestral_mamba'),
    #('open-mistral-7b', 'mistral_7b'),    
]
# actual prompt & prompt name for log file name
prompts_and_names = [
    (complete_code_prompt_basic, 'complete_code_prompt_basic'),            
    (complete_code_prompt,       'complete_code_prompt'),
    (complete_code_prompt_full,  'complete_code_prompt_full'),
    (complete_task_prompt_basic, 'complete_task_prompt_basic'),
    (complete_task_prompt,       'complete_task_prompt'),
    (complete_task_prompt_full,  'complete_task_prompt_full'), ]
# reflection prompts & their names for log file name
reflection_prompts_and_names = [
    ( reflection_prompt_basic, 'reflection_prompt_basic' ),
    ( reflection_prompt, 'reflection_prompt' ),
    ( reflection_prompt_full, 'reflection_prompt_full' ), ]
# temperature and top_k values and labels for file names
temperature_values = [
    (1.0,  'temperature1.0'),
    (0.75, 'temperature0.75'),
    (0.5,  'temperature0.5'), ]
top_p_values = [
    (1.0,  'topP1.0'),
    (0.75, 'topP0.75'),
    (0.5,  'topP0.5'), ]

model_idx            = 2
prompt_idx           = 3
relection_prompt_idx = 0
temperature_idx      = 0
top_p_idx            = 0

model_name, model_nickname     = models[ model_idx ]
my_prompt, my_prompt_label     = prompts_and_names[ prompt_idx ]
my_reflection_prompt, my_reflection_prompt_label = reflection_prompts_and_names[ relection_prompt_idx ]
TEMPERATURE, temperature_label = temperature_values[ temperature_idx ]
TOP_P, top_p_label             = top_p_values[ top_p_idx ]

SPECIAL_MESSAGE = f'Model temperature: {TEMPERATURE}. Model top_p: {TOP_P}. Model top_k: 50. Model dtype: torch.float32'

print(f'Model name:     {model_name}')
print(f'Model nickname: {model_nickname}')
print(f'Prompt name:    {my_prompt_label}')
print(f'Relfection prompt name: {my_reflection_prompt_label}')
print(f'Prompt:\n{my_prompt.strip().format("Task Description", "Test Cases")}\n')
print(f'Temperature: {TEMPERATURE}. Temperature label: {temperature_label}')
print(f'Top p: {TOP_P}. Top p label: {top_p_label}')
print(f'Special message: {SPECIAL_MESSAGE}')

Model name:     open-mistral-nemo
Model nickname: nemo
Prompt name:    complete_task_prompt_basic
Relfection prompt name: reflection_prompt_basic
Prompt:
Complete the following task:
Task Description.

Your output code must satisfy these tests:
Test Cases

Temperature: 1.0. Temperature label: temperature1.0
Top p: 1.0. Top p label: topP1.0
Special message: Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32


In [4]:
# save results
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
results_file   = f'logs_local/MBPP_{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_completions_{time_stamp}.jsonl'
log_file       = f'logs_local/logs/MBPP_{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_log_{time_stamp}.log'

# log results
for handler in logging.root.handlers[:]:            # overwrite any previous handlers with different formats
    logging.root.removeHandler(handler)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s  %(levelname)s  %(message)s',
    handlers=[
        logging.FileHandler(log_file),              # Log to a file
        logging.StreamHandler(sys.stdout)           # Log to console (default - sys.stderr (red background)
    ]
)

# print job config
logging.info('*****     JOB CONFIGURATION     *****')
logging.info(f'MODEL: {model_name}')
logging.info(f'MODEL NICKNAME: {model_nickname}')
logging.info(SPECIAL_MESSAGE)
logging.info(f'GENERATED CODE SAVED IN: {results_file}')
logging.info(f'THIS LOG FILE: {log_file}')
if my_prompt in [complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,]:
    logging.info(f'PROMPT:\n{my_prompt.lstrip().format("Starter Code")}')
else:
    logging.info(f'\nPROMPT:\n{my_prompt.format("Task Description", "Test Cases")}')
logging.info('*****     END OF JOB CONFIGURATION     *****')

2025-06-10 01:54:07,826  INFO  *****     JOB CONFIGURATION     *****
2025-06-10 01:54:07,827  INFO  MODEL: open-mistral-nemo
2025-06-10 01:54:07,827  INFO  MODEL NICKNAME: nemo
2025-06-10 01:54:07,828  INFO  Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32
2025-06-10 01:54:07,828  INFO  GENERATED CODE SAVED IN: logs_local/MBPP_nemo_complete_task_prompt_basic_temperature1.0_topP1.0_completions_20250610_015407_8260.jsonl
2025-06-10 01:54:07,829  INFO  THIS LOG FILE: logs_local/logs/MBPP_nemo_complete_task_prompt_basic_temperature1.0_topP1.0_log_20250610_015407_8260.log
2025-06-10 01:54:07,829  INFO  
PROMPT:

Complete the following task:
Task Description.

Your output code must satisfy these tests:
Test Cases

2025-06-10 01:54:07,830  INFO  *****     END OF JOB CONFIGURATION     *****


In [5]:
# read dataset
file = 'data/MBPP_Test.jsonl.gz'
tasks = read_problems(file)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 5:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: 11 <class 'str'>

task_id:
11

test:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

prompt:
Write a python function to remove first and last occurrence of a given character from the string.

canonical_solution:
def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s 

entry_point:
None


task_id: 12 <class 'str'>

task_id:
12

test:

assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]]
assert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]
assert sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1, 4], [6, 4, 3], [5, 8, 9]]

prompt:
Write a function to sort a given matrix in ascending order accordi

In [6]:
def api_call(model, prompt):
    ''' One API call to a Mistral model '''
    chat_response = client.chat.complete(
                model=model,
                messages=[{
                        "role": "user",
                        "content": prompt, 
                    }]
                )
    return chat_response.choices[0].message.content

def generate_response_with_backoff(model, prompt, max_retries=10, initial_delay=1, max_delay=16):
    '''
    Make API calls with exponential backoff on errors.
    Args:
        model (str): The model identifier.
        prompt (str): The prompt to send.
        max_retries (int): Maximum number of retries before giving up.
        initial_delay (int or float): Starting delay in seconds.
        max_delay (int or float): Maximum delay between retries.
    Returns:
        str: The chat response content OR f'Error generating a completion:{e}'
        if the max number of attemps has been reached
    '''
    retries = 0
    delay   = initial_delay
    while True:
        try:
            return api_call(model, prompt)
        except Exception as e:
            retries += 1
            if retries > max_retries:
                return f"Error generating a completion after {retries} retries:\n{e}"
            sleep_time = delay + random.uniform(0, 1)    # add jitter
            time.sleep(sleep_time)
            delay = min(delay * 2, max_delay)

In [7]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [8]:
input_prompt = "What is the capital of California?"
print(input_prompt)
print(generate_response_with_backoff(model_name, input_prompt))

What is the capital of California?
2025-06-10 01:54:09,121  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
The capital of California is Sacramento. It has been the state capital since 1854. Sacramento is known for its historic sites, such as the California State Capitol, the Crocker Art Museum, and Old Sacramento, a historic district along the Sacramento River. It's also the sixth-largest city in California, with a population of around 500,000 people.


In [9]:
num_samples_per_task = 1
completions  = []

for task_id, task_body in tasks.items():
    logging.info( task_id )
    #if task_id == 15:
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        
        # generate completion
        full_prompt = my_prompt.lstrip().format( task_body['prompt'], task_body['test'] )
        proposed_solution = generate_response_with_backoff( model_name, full_prompt )
        
        # improve solution
        new_full_prompt = my_reflection_prompt.lstrip().format(full_prompt, proposed_solution)
        improved_solution = generate_response_with_backoff( model_name, new_full_prompt )        
        
        # check if improved solution is a Python function. If not, use the previously proposed solution
        cleaned_improved_solution = clean_code_light(improved_solution).strip()
        if is_function(cleaned_improved_solution):
            final_solution = improved_solution
        else:
            final_solution = proposed_solution
            
        # save and log the results
        completions.append( {'task_id': task_id, 'completion': final_solution} )
        write_jsonl(results_file, completions)
        logging.info('NEW REFLECTION PROMPT:\n' + new_full_prompt + '\n')
        logging.info('IMPROVED COMPLETION:\n' + final_solution + '\n')
        logging.info(f"Time elapsed including backoff: {(time.time() - start_time):.4f} seconds\n")

2025-06-10 01:54:09,145  INFO  11
2025-06-10 01:54:13,011  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 01:54:14,622  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 01:54:14,630  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a python function to remove first and last occurrence of a given character from the string..

Your output code must satisfy these tests:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"


PROPOSED SOLUTION:
Here's a Python function that removes the first and last occurr

2025-06-10 01:54:24,928  INFO  Time elapsed including backoff: 5.5793 seconds

2025-06-10 01:54:24,929  INFO  14
2025-06-10 01:54:27,405  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 01:54:29,464  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 01:54:29,471  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a python function to find the volume of a triangular prism..

Your output code must satisfy these tests:

assert find_Volume(10,8,6) == 240
assert find_Volume(3,2,2) == 6
assert find_Volume(1,2,1) == 1


PROPOSED SOLUTION:
Here's a Python function that calcul

2025-06-10 01:54:41,473  INFO  Time elapsed including backoff: 6.0034 seconds

2025-06-10 01:54:41,474  INFO  17
2025-06-10 01:54:42,338  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 01:54:43,497  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 01:54:43,509  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a function to find the perimeter of a square..

Your output code must satisfy these tests:

assert square_perimeter(10)==40
assert square_perimeter(5)==20
assert square_perimeter(4)==16


PROPOSED SOLUTION:
Here's a simple function in Python that calculates t

2025-06-10 01:54:56,954  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and correct. It correctly implements the Woodall number checking algorithm using the formula `2^(2k-1) + 1`. The solution also includes test cases to validate the function's correctness.

Here's the proposed solution with some minor formatting improvements:

```python
def is_woodall(n):
    # Woodall numbers are of the form 2^(2k-1) + 1
    # where k is a positive integer.
    # We can check if n is a Woodall number by checking if it is of this form.
    return (n - 1) % 2 == 0 and (n - 1) % 4 == 0

# Test cases
assert is_woodall(383) == True
assert is_woodall(254) == False
assert is_woodall(200) == False
```

Since the proposed solution is already optimal and correct, I will return it as is.

2025-06-10 01:54:56,955  INFO  Time elapsed including backoff: 3.9030 seconds

2025-06-10 01:54:56,956  INFO  21
2025-06-10 01:54:58,820  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completion

2025-06-10 01:55:44,449  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 01:55:44,461  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a function to convert the given binary number to its decimal equivalent..

Your output code must satisfy these tests:

assert binary_to_decimal(100) == 4
assert binary_to_decimal(1011) == 11
assert binary_to_decimal(1101101) == 109


PROPOSED SOLUTION:
Here is a Python solution for the problem:

```python
def binary_to_decimal(binary):
    binary = str(binary)
    decimal, i, n = 0, 0, 0
    while(binary != 0):
        dec = binary % 10
        decimal = decimal + dec * pow(2, i

...

2025-06-10 02:56:58,107  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and correct. It uses the built-in `heapq.nsmallest` function which is designed to efficiently find the smallest `k` elements from a list. The function first converts the list into a heap, then pops the smallest element `k` times, which is an efficient operation for heaps.

Here's the proposed solution again for reference:

```python
import heapq

def heap_queue_smallest(nums, k):
    return heapq.nsmallest(k, nums)

assert heap_queue_smallest( [25, 35, 22, 85, 14, 65, 75, 25, 58],3)==[14, 22, 25]
assert heap_queue_smallest( [25, 35, 22, 85, 14, 65, 75, 25, 58],2)==[14, 22]
assert heap_queue_smallest( [25, 35, 22, 85, 14, 65, 75, 22, 58],5)==[14, 22, 22, 25, 35]
```

Since the proposed solution is already optimal and correct, I will return it as is.

2025-06-10 02:56:58,108  INFO  Time elapsed including backoff: 3.8970 seconds

2025-06-10 02:56:58,108  INFO  497
2025-06-10 02:57:48,693  INFO  HT

2025-06-10 02:58:36,548  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 02:58:36,568  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a function to concatenate all elements of the given list into a string..

Your output code must satisfy these tests:

assert concatenate_elements(['hello','there','have','a','rocky','day'] ) == '  hello there have a rocky day'
assert concatenate_elements([ 'Hi', 'there', 'How','are', 'you'] ) == '  Hi there How are you'
assert concatenate_elements([ 'Part', 'of', 'the','journey', 'is', 'end'] ) == '  Part of the journey is end'


PROPOSED SOLUTION:
Here's a simple Python functio

2025-06-10 02:58:46,491  INFO  IMPROVED COMPLETION:
The proposed solution is already optimal and correct. It uses list comprehension, which is a concise and efficient way to create lists in Python. The function iterates over the input list in steps of 2, sums the two numbers at each step, and adds the result to the output list. This solution satisfies all the given test cases.

Here's the proposed solution again for reference:

```python
def add_consecutive_nums(lst):
    return [sum(lst[i:i+2]) for i in range(0, len(lst), 2)]

# Test cases
assert add_consecutive_nums([1, 1, 3, 4, 4, 5, 6, 7])==[2, 4, 7, 8, 9, 11, 13]
assert add_consecutive_nums([4, 5, 8, 9, 6, 10])==[9, 13, 17, 15, 16]
assert add_consecutive_nums([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])==[3, 5, 7, 9, 11, 13, 15, 17, 19]
```

Since the proposed solution is already optimal and correct, there's no need to generate an optimized version.

2025-06-10 02:58:46,491  INFO  Time elapsed including backoff: 4.5339 seconds

2025-06-10 02:

2025-06-10 02:58:55,462  INFO  Time elapsed including backoff: 2.3065 seconds

2025-06-10 02:58:55,463  INFO  507
2025-06-10 02:58:57,075  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 02:58:58,976  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 02:58:58,996  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a function to remove specific words from a given list..

Your output code must satisfy these tests:

assert remove_words(['red', 'green', 'blue', 'white', 'black', 'orange'],['white', 'orange'])==['red', 'green', 'blue', 'black']
assert remove_words(['red', 

2025-06-10 02:59:04,810  INFO  Time elapsed including backoff: 2.7216 seconds

2025-06-10 02:59:04,810  INFO  510
2025-06-10 02:59:08,003  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 02:59:10,683  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-10 02:59:10,704  INFO  NEW REFLECTION PROMPT:
1. Read and understand the REQUIREMENTS and the PROPOSED SOLUTION listed below.
2. Analyze the PROPOSED SOLUTION for any errors, inefficiencies or inconsistencies.
3. Generate ONE optimized version of the PROPOSED SOLUTION.
4. If the PROPOSED SOLUTION is already optimal, return the PROPOSED SOLUTION.

REQUIREMENTS:
Complete the following task:
Write a function to find the number of subsequences having product smaller than k for the given non negative array..

Your output code must satisfy these tests:

assert no_of_subsequences([1,2,3,4], 10) == 11
assert no_of_subsequences([4,8,7,2], 50) == 9
assert no

In [10]:
unknown_variable

NameError: name 'unknown_variable' is not defined

# Appendix

In [ ]:
# read a jsonl file
file_name = 'logs/mistral_7b_samples_20241201_005244_0462.jsonl'
data = read_jsonl(file_name)

In [ ]:
data[0]['completion']

In [ ]:
# modify and write back
data2 = []
for i in data:
    new_i = dict()
    new_i['task_id'] = i['task_id']
    new_i['completion'] = add_import(i['completion'])
    data2.append(new_i)

In [ ]:
file_name = 'logs/mistral_7b_samples_20241201_005244_0462_v2.jsonl'
write_jsonl(file_name, data2)